## About this test
This test shall take as input a BluePyOpt optimized output file. The validation tests run on the hoc template specified as "best_cell" in the `meta.json` file. There are a total of four different tests in this use case:

1. __Somatic Features Test__:  _(for pyramidal cells and interneurons)_<br/>
    The Somatic Features Test can be used for both pyramidal cells and interneurons. It evaluates the model against various eFEL features under somatic current injection of varying amplitudes.<br/><br/>

2. __Depolarization Block Test__:  _(for pyramidal cells)_<br/>
    The Depolarization Block Test aims to determine whether the model enters depolarization block to prolonged, high intensity somatic current stimulus. It compares the current intensity to which the model fires the maximum number of action potentials, the current intensity before the model enters depolarization block (the two should be equal) and the equilibrium potential during depolarization block to the experimental data of Bianchi et al. 2012 (http://dx.doi.org/10.1007/s10827-012-0383-y).<br/><br/>

3. __Back-Propagating AP Test__:  _(for pyramidal cells)_<br/>
    The Back-Propagating AP Test evaluates the mode and efficacy of back-propagating action potentials on the apical trunk in locations of different distances from the soma. The amplitude of the first and last AP of around 15 Hz train is compared to experimental data from Golding et al. 2001 (https://doi.org/10.1152/jn.2001.86.6.2998).<br/><br/>

4. __PSP Attenuation Test__:  _(for pyramidal cells)_<br/>
    The PSP Attenuation Test evaluates how much the post synaptic potential (using EPSC stimulus) attenuates from the dendrite (different distances) to the soma. The soma/dendrite attenuation is compared to data from Magee & Cook 2000 (http://dx.doi.org/10.1038/78800).<br/>


The results are registered on the HBP Validation Framework app. If an instance of the Model Catalog and Validation Framework are not found in the current Collab, then these are created. Additionally, a test report is generated and this can be viewed within the Jupyter notebook, or downloaded.

In [ ]:
import os
import pkg_resources
from pkg_resources import parse_version

req_packages = {    
                    "hbp_service_client"       : {"min_version": "1.1.0",  "install_version": "1.1.0"},
                    "sciunit"                  : {"min_version": "",       "install_version": "git://github.com/scidash/sciunit.git@9e61c060cd8e5c6d7bce675d37c398987a9491c5"},
                    "neuronunit"               : {"min_version": "0.1.8.2","install_version": "0.1.8.2"},
                    "hippounit"                : {"min_version": "1.1",    "install_version": "1.1"},
                    "bluepyopt"                : {"min_version": "1.6.22", "install_version": "1.6.42"},                    
                    "hbp_validation_framework" : {"min_version": "0.5.4",  "install_version": "0.5.4"},
                    "numpy"                    : {"min_version": "1.16.2", "install_version": "1.16.2"},    
                    "fpdf"                     : {"min_version": "1.7.2",  "install_version": "1.7.2"},
                    "PyPDF2"                   : {"min_version": "1.26.0", "install_version": "1.26.0"}
               }

def install_req_packages():
    # currently handles installations via PyPI and GitHub
    for pkg_name, pkg_vinfo in req_packages.items():
        print("Checking for package: {}".format(pkg_name))
        try:
            pkg_resources.get_distribution(pkg_name)
#             if pkg_name == "sciunit":
#                 !pip uninstall -y sciunit               
            current_version = parse_version(pkg_resources.get_distribution(pkg_name).version)
            if not pkg_vinfo["min_version"] or current_version < parse_version(pkg_vinfo["min_version"]) or current_version > parse_version(pkg_vinfo["install_version"]):
                    print("\t{}: current version = {}".format(pkg_name, current_version))
                    print("\tAnother version of {} will be installed.".format(pkg_name))
                    raise
        except:         
            if "github.com" in pkg_vinfo["install_version"]:
                os.system("pip install --quiet --no-cache-dir --upgrade git+{}".format(pkg_vinfo["install_version"]))                
            else:
                os.system("pip install --quiet --no-cache-dir --upgrade {}=={}".format(pkg_name, pkg_vinfo["install_version"]))                                
                
            if pkg_name == "hbp_service_client":
                from IPython.display import HTML
                display(HTML('''<script>window.requestAnimationFrame(() => { Jupyter.notebook.kernel.restart(); \
                Jupyter.notebook.dirty = false; window.location.reload(); })</script>'''))
                
install_req_packages()                 
        
import sciunit
import bluepyopt.ephys as ephys
import hippounit
from hippounit.utils import ModelLoader_BPO
from hbp_validation_framework import utils, TestLibrary, ModelCatalog

import json
import requests
import urllib2
import StringIO
import zipfile
import collections
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import warnings
warnings.filterwarnings('ignore')

### Check if Model Catalog and Validation Framework Apps Exist in Collab
If the notebook is run inside a Collab, we check if an instance of the Model Catalog and Validation Framework apps exist in the current Collab. If not, we add an instance of each (this will be reflected in the Collab's navigation panel, possibly on reloading the page).

NOTE: **HBP_USERNAME** is an optional parameter when the notebook is being run inside the Collaboratory. The notebook can automatically identify your username in this scenario. This parameter needs to be specified if a user wishes to download the notebook and run it locally. Another potential (less likely) reason for specifying this (even within the Collaboratory) is in dealing with access permissions (wanting to run the test with different credentials).

NOTE: Even if this notebook is not run inside a Collab, the following cell needs to be executed. It will identify if environment and manage accordingly. When not run inside a Collab, it will simply setup parameters required for the test, and not attempt to create new apps.

In [ ]:
# your HBP username; not essential if running inside the Collaboratory
HBP_USERNAME = ""
testLibrary = TestLibrary(username=HBP_USERNAME)
modelCatalog = ModelCatalog.from_existing(testLibrary)

try:
    collab_path = get_collab_storage_path()
    collab_id = collab_path[1:] # this might fail for very old Collabs which use name instead of Collab ID
except:
    # not run inside Collaboratory
    print("\nPlease enter a Collab ID where you wish to store the results:")
    print("E.g.: 8123")
    print("Note: you should be a member of this Collab!")
    collab_id = input()
    if not isinstance(collab_id, int):
        raise ValueError("Possibly invalid Collab ID: {}. Numeric input expected!".format(collab_id))    

# check if apps exist; if not then create them
MCapp_navID = modelCatalog.exists_in_collab_else_create(collab_id)
modelCatalog.set_app_config(collab_id=collab_id, app_id=MCapp_navID, only_if_new="True")
VFapp_navID = testLibrary.exists_in_collab_else_create(collab_id)
testLibrary.set_app_config(collab_id=collab_id, app_id=VFapp_navID, only_if_new="True")

### Model Selection
The user is given an option to choose from existing models sourced from https://object.cscs.ch/v1/AUTH_c0a333ecf7c045809321ce9d9ecdfdea/hippocampus_optimization/ (see: `/optimizations/`). Only models available here are shown as existing models.

In [ ]:
base_path = "https://object.cscs.ch/v1/AUTH_c0a333ecf7c045809321ce9d9ecdfdea/hippocampus_optimization/"
repoItems = urllib2.urlopen(base_path).read().split("\n")    
entries = []
for item in repoItems:
    if item.endswith(".zip"):
        entries.append(item.rsplit("/",1)[1][:-4])
print("\nExisting models are listed below:")
halfLen = int(len(entries)/2)
for i, each in enumerate(zip(entries,entries[halfLen:]),start=1):
    print ("{0:>3}. {2:>30} \t {1:>3}. {3:>30}".format(i,i+halfLen,*each))
print("\nEnter the # of required model: ")
choice = input()
if choice <= len(entries):        
    model_name = entries[choice-1]        
    file_path = os.path.join(base_path,"optimizations", model_name, model_name + ".zip")
else:
    raise ValueError("Invalid entry for model choice!")

try:        
    response = requests.get(file_path)
    zip_ref = zipfile.ZipFile(StringIO.StringIO(response.content))
    zip_ref.extractall()
    model_path = os.path.join(os.getcwd(),model_name)

    meta_info = requests.get(file_path.replace(".zip", "_meta.json"))    
    if meta_info.status_code == 200:
        with open(os.path.join(model_path, model_name+"_meta.json"), 'w') as f:
            json.dump(meta_info.json(), f)
    
    model_image_url = file_path.replace(model_name+".zip", "_".join(model_name.split("_")[3:-1])+"_morph.jpeg")
    model_image = requests.get(model_image_url)    
    model_image_localPath = None
    if model_image.status_code == 200:        
        model_image_localPath = os.path.join(model_path, model_name.split("_")[3]+"_morph.jpg")
        with open(model_image_localPath, 'wb') as f:
            f.write(model_image.content)         
        print "\nModel Morphology:"
        img = mpimg.imread(model_image_localPath)
        imgplot = plt.imshow(img)
        plt.show()
except:
    raise IOError("Model url = {} is invalid!".format(file_path))

### Gather Additional Info
Some of the tests are applicable only for pyramidal cells, while others are more generic (also applicable for interneurons). Hence it is essential to identify the cell type: _pyramidal_ or _interneuron_ <br />
The usecase tries to identify the cell type from the model name. If this isn't possible, the user is prompted to specify the cell type.

In [ ]:
# will try to determine cell type from file name; if not then ask user
# current options: msn_d1, msn_d2
if "_pyr_" in model_name:
    cell_type = "Pyramidal Cell" 
elif "_int_" in model_name:
    cell_type = "Interneuron"
else:
    print("\nPlease enter the cell_type: ")
    options = ["Pyramidal Cell", "Interneuron"]
    for i, each in enumerate(options,start=1):
        print("\t{}. {}".format(i,each))
    print("Enter the # of cell_type: ")
    choice = input()
    cell_type = options[choice-1]
print("Cell Type = {}".format(cell_type))

### Specifying model from ModelCatalog; Update info if new model entry
For a model to use the validation framework, it first has to be registered on the _Model Catalog_ app (see Collab's Navigation panel; select `Model Catalog`) <br />

If the model has already been registered on the model catalog, then you simply need to specify the `model_uuid`. <br /> E.g. c7748201-5551-4d56-bc31-f7feec5e5798 <br />

If this is to be registered as a new model, then you are required to update the cell below to alter the model metadata as required. See [here](http://hbp-validation-client.readthedocs.io/en/master/#hbp_validation_framework.ModelCatalog.register_model) for detailed info on the various parameters. Once a new model is registered on the model catalog, it will show you the corresponding `model_uuid`. This can be used for future tests to link the results with the same model entry.

In [ ]:
print("\nDo you wish to use an existing `model_uuid` (this will register results against an existing model entry)?")
print("Note: `model_uuid` is obtained from the HBP Validation Framework app.")
print("If not specifying `model_uuid`, then a new model entry would be registered.")
print("Note: If registering a new model, please verify the default model metadata in this Jupyter notebook!")
print("Enter: y/n")
choice = raw_input().lower()
valid_choices = {"yes": True, "y": True, "no": False, "n": False}
if valid_choices[choice]:
    print("\nPlease enter a valid `model_uuid`: ")
    model_uuid = raw_input()
    # Verify the user specified `model_uuid` is valid
    try:
        modelCatalog.get_model(model_id=model_uuid)
    except:
        raise ValueError("Specified model_uuid = {} is invalid!".format(model_uuid))
else:
    # Create a new model entry in the ModelCatalog; metadata to be used for registering model
    model_uuid = modelCatalog.register_model(
                                app_id=str(MCapp_navID),
                                name=model_name,
                                # alias="", # optional; ensure uniqueness                                
                                author="Rosanna Migliore",
                                organization="",
                                private=False,
                                cell_type=cell_type,
                                model_scope="single cell",
                                abstraction_level="spiking neurons",
                                brain_region="hippocampus",
                                species="Rattus norvegicus",
                                description="This model is being used to demonstrate use of the Validation Service")
    model_uuid = str(model_uuid)
    if model_image_localPath:
        modelCatalog.add_model_image(model_id=model_uuid,
                                      url=model_image_url,
                                      caption="Model Morphology")
    print("\nModel registered on the HBP Validation Framework's Model Catalog!")
    print("Note: you can specify the below `model_uuid` for future runs of this test.")
print("\nmodel_uuid = {}".format(model_uuid))
print("Link to ModelCatalog app:")
print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=model.{}".format(str(collab_id),str(MCapp_navID),model_uuid))

### Selecting the Validation Test

In [ ]:
test_list = {"Somatic Features Test" : ["Pyramidal Cell", "Interneuron"],
             "Back-Propagating AP Test" : ["Pyramidal Cell"], 
             "PSP Attenuation Test" : ["Pyramidal Cell"], 
             "Depolarization Block Test" : ["Pyramidal Cell"]}
valid_test_list = [key for key,val in test_list.items() if cell_type in val]
    
print("\nPlease choose from the following available tests for your cell type: ")
for i, each in enumerate(valid_test_list,start=1):
        print("\t{}. {}".format(i,each))
print("Note: you may specify multiple tests: ")
print("Example inputs: 1, 4, [1,4], all")
             
test_entry = raw_input().lower()
try:
    if test_entry == "all":
        select_test_list = valid_test_list
    else:    
        if isinstance(eval(test_entry), list):
            test_entry = eval(test_entry)
            test_entry = [x - 1 for x in test_entry] # menu numbering was from 1            
            select_test_list = [valid_test_list[i] for i in test_entry]
        elif isinstance(eval(test_entry), int):
            test_entry = eval(test_entry) - 1 # menu numbering was from 1
            select_test_list = [valid_test_list[test_entry]]
        else:
            raise ValueError("Invalid entry for test selection!")
except ValueError:
    print("Invalid entry for test selection!")

### Instantiating the model; Running Model Validation for Selected Tests
The usecase will run validations on the specified model for each of the specified tests. At the end of the test, the user is provided with a textual summary of the score and the path to related output files generated by the tests. These and other details can be viewed in the Validation Framework app (see Collab's Navigation panel; select Validation Framework).

In [ ]:
cell_model = ModelLoader_BPO(name=model_name, model_dir=model_path, SomaSecList_name = "somatic")
cell_model.model_uuid = model_uuid
cell_model.source = file_path
cell_model.model_version = "1.0"

result_uuids = []

for test in select_test_list:
    if test == "Somatic Features Test":
        if 'CA1_pyr_cACpyr' in model_name:                          
            ttype = "CA1_pyr_cACpyr"        
        elif 'CA1_int_bAC' in model_name:
            ttype = "CA1_int_bAC"
        elif 'CA1_int_cAC' in model_name:
            ttype = "CA1_int_cAC"
        elif 'CA1_int_cNAC' in model_name:
            ttype = "CA1_int_cNAC"
        stim_file = pkg_resources.resource_filename("hippounit", "tests/stimuli/somafeat_stim/stim_" + ttype + ".json")
        with open(stim_file, 'r') as f:
            config = json.load(f, object_pairs_hook=collections.OrderedDict)
        result_id, score = utils.run_test(username=HBP_USERNAME, environment="production", model=cell_model, test_alias="hippo_somafeat_"+ttype, test_version="1.0", ttype=ttype, storage_collab_id=collab_id, register_result=True, config=config)            
    
    elif test == "Back-Propagating AP Test":
        stim_file = pkg_resources.resource_filename("hippounit", "tests/stimuli/bAP_stim/stim_bAP_test.json")
        with open(stim_file, 'r') as f:
            config = json.load(f, object_pairs_hook=collections.OrderedDict)
        result_id, score = utils.run_test(username=HBP_USERNAME, environment="production", model=cell_model, test_alias="hippo_ca1_bap", test_version="1.0", storage_collab_id=collab_id, register_result=True, config=config)            
        
    elif test == "PSP Attenuation Test":
        stim_file = pkg_resources.resource_filename("hippounit", "tests/stimuli/PSP_attenuation_stim/stim_PSP_attenuation_test.json")
        with open(stim_file, 'r') as f:
            config = json.load(f, object_pairs_hook=collections.OrderedDict)
        result_id, score = utils.run_test(username=HBP_USERNAME, environment="production", model=cell_model, test_alias="hippo_ca1_psp_attenuation", test_version="1.0", storage_collab_id=collab_id, register_result=True, config=config)
        
    elif test == "Depolarization Block Test":
        result_id, score = utils.run_test(username=HBP_USERNAME, environment="production", model=cell_model, test_alias="hippo_ca1_depolblock", test_version="1.0", storage_collab_id=collab_id, register_result=True)            
               
    result_uuids.append(result_id)
    
print("The result(s) can be viewed in the HBP Validation Framework app. Direct link(s):")
for result_uuid in result_uuids:
    print("https://collab.humanbrainproject.eu/#/collab/{}/nav/{}?state=result.{}".format(str(collab_id),str(VFapp_navID), result_uuid))

### Generate Report
The validation framework can generate a PDF report for all the successfully completed tests. The user is prompted whether such a report should be generated for the current tests. If asked to generate, the location to the generated PDF is indicated.

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        valid_uuids, report_path = utils.generate_report(environment="production", result_list=result_uuids, only_combined=True, client_obj=modelCatalog)    

### View Report Inside Jupyter Notebook
The PDF report created in the above cell is displayed within the Jupyter notebook. This can also be downloaded by clicking the download button inside the display frame.

In [ ]:
print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))